In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# df

In [3]:
import copy
import torch

# Make use of a GPU or MPS (Apple) if one is available.  (see module 3.2)
import torch
has_mps = torch.backends.mps.is_built()
device = "mps" if has_mps else "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")


# Early stopping (see module 3.4)
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0, restore_best_weights=True):
        self.patience = patience
        self.min_delta = min_delta
        self.restore_best_weights = restore_best_weights
        self.best_model = None
        self.best_loss = None
        self.counter = 0
        self.status = ""

    def __call__(self, model, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
            self.best_model = copy.deepcopy(model.state_dict())
        elif self.best_loss - val_loss >= self.min_delta:
            self.best_model = copy.deepcopy(model.state_dict())
            self.best_loss = val_loss
            self.counter = 0
            self.status = f"Improvement found, counter reset to {self.counter}"
        else:
            self.counter += 1
            self.status = f"No improvement in the last {self.counter} epochs"
            if self.counter >= self.patience:
                self.status = f"Early stopping triggered after {self.counter} epochs."
                if self.restore_best_weights:
                    model.load_state_dict(self.best_model)
                return True
        return False

Using device: cuda


In [4]:
# import pandas as pd
# from scipy.stats import zscore
# from sklearn.model_selection import train_test_split

# # Read the data set
# df = pd.read_csv(
#     "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
#     na_values=['NA','?'])

# # Generate dummies for job
# df = pd.concat([df,pd.get_dummies(df['job'],prefix="job",dtype=int)],axis=1)
# df.drop('job', axis=1, inplace=True)

# # Generate dummies for area
# df = pd.concat([df,pd.get_dummies(df['area'],prefix="area",dtype=int)],axis=1)
# df.drop('area', axis=1, inplace=True)

# # Generate dummies for product
# df = pd.concat([df,pd.get_dummies(df['product'],prefix="product",dtype=int)],axis=1)
# df.drop('product', axis=1, inplace=True)

# # Missing values for income
# med = df['income'].median()
# df['income'] = df['income'].fillna(med)

# # Standardize ranges
# df['income'] = zscore(df['income'])
# df['aspect'] = zscore(df['aspect'])
# df['save_rate'] = zscore(df['save_rate'])
# df['subscriptions'] = zscore(df['subscriptions'])

# df

In [5]:
# import pandas as pd
# import torch
# import torch.nn as nn
# import torch.optim as optim
# from torch.utils.data import DataLoader, TensorDataset
# from sklearn.model_selection import KFold
# from sklearn.preprocessing import StandardScaler

# # Convert to PyTorch Tensors
# x_columns = df.columns.drop(['age', 'id'])
# x = torch.tensor(df[x_columns].values, dtype=torch.float32, device=device)
# y = torch.tensor(df['age'].values, dtype=torch.float32, device=device).view(-1, 1)

# # Set random seed for reproducibility
# torch.manual_seed(42)

# # Cross-Validate
# kf = KFold(n_splits=5, shuffle=True, random_state=42)

# # Early stopping parameters
# patience = 10

# fold = 0
# for train_idx, test_idx in kf.split(x):
#     fold += 1
#     print(f"Fold #{fold}")

#     x_train, x_test = x[train_idx], x[test_idx]
#     y_train, y_test = y[train_idx], y[test_idx]

#     # PyTorch DataLoader
#     train_dataset = TensorDataset(x_train, y_train)
#     train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

#     # Create the model and optimizer
#     model = nn.Sequential(
#         nn.Linear(x.shape[1], 20),
#         nn.ReLU(),
#         nn.Linear(20, 10),
#         nn.ReLU(),
#         nn.Linear(10, 1)
#     )
#     model = torch.compile(model,backend="aot_eager").to(device)


#     optimizer = optim.Adam(model.parameters(), lr=0.001)
#     loss_fn = nn.MSELoss()

#     # Early Stopping variables
#     best_loss = float('inf')
#     early_stopping_counter = 0

#     # Training loop
#     EPOCHS = 500
#     epoch = 0
#     done = False
#     es = EarlyStopping()

#     while not done and epoch<EPOCHS:
#         epoch += 1
#         model.train()
#         for x_batch, y_batch in train_loader:
#             optimizer.zero_grad()
#             output = model(x_batch)
#             loss = loss_fn(output, y_batch)
#             loss.backward()
#             optimizer.step()

#         # Validation
#         model.eval()
#         with torch.no_grad():
#             val_output = model(x_test)
#             val_loss = loss_fn(val_output, y_test)

#         if es(model, val_loss):
#             done = True

#     print(f"Epoch {epoch}/{EPOCHS}, Validation Loss: "
#       f"{val_loss.item()}, {es.status}")

# # Final evaluation
# model.eval()
# with torch.no_grad():
#     oos_pred = model(x_test)
# score = torch.sqrt(loss_fn(oos_pred, y_test)).item()
# print(f"Fold score (RMSE): {score}")

In [6]:
# x_test.shape

In [7]:
df = pd.read_csv(
    "https://data.heatonresearch.com/data/t81-558/jh-simple-dataset.csv",
    na_values=['NA','?'])
df
# y.shape

,id,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,1,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,2,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,3,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,4,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,5,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,1996,vv,c,51017.0,38.233333,1,5.454545,34,14.013489,41,0.881890,0.744094,0.104838,b
1996,1997,kl,d,26576.0,33.358333,2,3.632069,20,8.380497,38,0.944882,0.877953,0.063851,a
1997,1998,kl,d,28595.0,39.425000,3,7.168218,99,4.626950,36,0.759843,0.744094,0.098703,f
1998,1999,qp,c,67949.0,5.733333,0,8.936292,26,3.281439,46,0.909449,0.598425,0.117803,c


In [8]:
# oos_pred = model(x_test)
# oos_pred.shape

In [9]:
df.drop("id",axis=1,inplace=True)


In [10]:
df

,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,product
0,vv,c,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,b
1,kd,c,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,c
2,pe,c,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,b
3,11,c,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,b
4,kl,d,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,a
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,vv,c,51017.0,38.233333,1,5.454545,34,14.013489,41,0.881890,0.744094,0.104838,b
1996,kl,d,26576.0,33.358333,2,3.632069,20,8.380497,38,0.944882,0.877953,0.063851,a
1997,kl,d,28595.0,39.425000,3,7.168218,99,4.626950,36,0.759843,0.744094,0.098703,f
1998,qp,c,67949.0,5.733333,0,8.936292,26,3.281439,46,0.909449,0.598425,0.117803,c


In [11]:
# import pandas as pd
# import numpy as np
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from xgboost import XGBClassifier
import matplotlib.pyplot as plt
import seaborn as sns

le=LabelEncoder()

df["job"]=le.fit_transform(df["job"])
df["area"]=le.fit_transform(df["area"])
df["preduct"]=le.fit_transform(df["product"])
# df["job"]=le.fit_transform(df["job"])
df.drop("product",inplace=True,axis=1)
df

,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,age,pop_dense,retail_dense,crime,preduct
0,31,2,50876.0,13.100000,1,9.017895,35,11.738935,49,0.885827,0.492126,0.071100,1
1,13,2,60369.0,18.625000,2,7.766643,59,6.805396,51,0.874016,0.342520,0.400809,2
2,23,2,55126.0,34.766667,1,3.632069,6,13.671772,44,0.944882,0.724409,0.207723,1
3,0,2,51690.0,15.808333,1,5.372942,16,4.333286,50,0.889764,0.444882,0.361216,1
4,15,3,28347.0,40.941667,3,3.822477,20,5.967121,38,0.744094,0.661417,0.068033,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,31,2,51017.0,38.233333,1,5.454545,34,14.013489,41,0.881890,0.744094,0.104838,1
1996,15,3,26576.0,33.358333,2,3.632069,20,8.380497,38,0.944882,0.877953,0.063851,0
1997,15,3,28595.0,39.425000,3,7.168218,99,4.626950,36,0.759843,0.744094,0.098703,5
1998,27,2,67949.0,5.733333,0,8.936292,26,3.281439,46,0.909449,0.598425,0.117803,2


In [12]:
df.info()
med = df["income"].median()
df["income"]=df["income"].fillna(med)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   job             2000 non-null   int64  
 1   area            2000 non-null   int64  
 2   income          1941 non-null   float64
 3   aspect          2000 non-null   float64
 4   subscriptions   2000 non-null   int64  
 5   dist_healthy    2000 non-null   float64
 6   save_rate       2000 non-null   int64  
 7   dist_unhealthy  2000 non-null   float64
 8   age             2000 non-null   int64  
 9   pop_dense       2000 non-null   float64
 10  retail_dense    2000 non-null   float64
 11  crime           2000 non-null   float64
 12  preduct         2000 non-null   int64  
dtypes: float64(7), int64(6)
memory usage: 203.3 KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 13 columns):
 #   Column          Non-Null Count  Dtype  
---  ------   

In [13]:
from scipy.stats import zscore
df["income"]=zscore(df["income"])
df["aspect"]=zscore(df["aspect"])
df['save_rate'] = zscore(df['save_rate'])
df['subscriptions'] = zscore(df['subscriptions'])

In [14]:
y=df["age"]
X=df.drop("age",axis=1)
X
y
X

,job,area,income,aspect,subscriptions,dist_healthy,save_rate,dist_unhealthy,pop_dense,retail_dense,crime,preduct
0,31,2,-0.607550,-0.664918,-0.208449,9.017895,-0.215764,11.738935,0.885827,0.492126,0.071100,1
1,13,2,0.338053,-0.207748,0.839031,7.766643,0.196869,6.805396,0.874016,0.342520,0.400809,2
2,23,2,-0.184205,1.127906,-0.208449,3.632069,-0.714362,13.671772,0.944882,0.724409,0.207723,1
3,0,2,-0.526467,-0.440815,-0.208449,5.372942,-0.542432,4.333286,0.889764,0.444882,0.361216,1
4,15,3,-2.851675,1.638861,1.886511,3.822477,-0.473660,5.967121,0.744094,0.661417,0.068033,0
...,...,...,...,...,...,...,...,...,...,...,...,...
1995,31,2,-0.593504,1.414758,-0.208449,5.454545,-0.232957,14.013489,0.881890,0.744094,0.104838,1
1996,15,3,-3.028085,1.011372,0.839031,3.632069,-0.473660,8.380497,0.944882,0.877953,0.063851,0
1997,15,3,-2.826971,1.513363,1.886511,7.168218,0.884591,4.626950,0.759843,0.744094,0.098703,5
1998,27,2,1.093101,-1.274478,-1.255928,8.936292,-0.370502,3.281439,0.909449,0.598425,0.117803,2


##  Convert Array/DataFrame to Pytorch Tensor
<!-- # 🟥 This is a Very Big Header (H1) -->

In [15]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [16]:
torch.manual_seed(42)
X_tensor=torch.tensor(X.values,dtype=torch.float32,device=device)
device
y_tensor=torch.tensor(y.values,dtype=torch.float32,device=device).view(-1,1)

kf=KFold(n_splits=5,shuffle=True,random_state=42)
kf

patience=10

fold=0

for train_index,test_index in kf.split(X_tensor):
    fold+=1
    print(f"Fold {fold}")
    X_train, X_test= X_tensor[train_index], X_tensor[test_index]
    y_train, y_test= y_tensor[train_index], y_tensor[test_index]



    train_dataset=TensorDataset(X_train,y_train)
    train_loader=DataLoader(train_dataset, batch_size=32, shuffle=True)


    model=nn.Sequential(
        nn.Linear(12,20),
        nn.ReLU(),
        nn.Linear(20,20),
        nn.ReLU(),
        nn.Linear(20,1),
        nn.ReLU()
    )
    model = torch.compile(model,backend="aot_eager").to(device)



    loss_fn=nn.MSELoss()
    optimizer=torch.optim.Adam(params=model.parameters(),lr=0.01)
    # Early Stopping variables
    best_loss = float('inf')
    early_stopping_counter = 0

    Epochs=500
    
    # Training loop
    EPOCHS = 500
    epoch = 0
    done = False
    es = EarlyStopping()


    while not done and epoch<EPOCHS:
        epoch += 1
        model.train()
        for x_batch, y_batch in train_loader:
            optimizer.zero_grad()
            output = model(x_batch)
            loss = loss_fn(output, y_batch)
            loss.backward()
            optimizer.step()

        # Validation
        model.eval()
        with torch.inference_mode():
            val_output = model(X_test)
            val_loss = loss_fn(val_output, y_test)

        if es(model, val_loss):
            done = True

    print(f"Epoch {epoch}/{EPOCHS}, Validation Loss: "
      f"{val_loss.item()}, {es.status}")

# Final evaluation
model.eval()
with torch.inference_mode():
    oos_pred = model(X_test)
score = torch.sqrt(loss_fn(oos_pred, y_test)).item()
print(f"Fold score (RMSE): {score}")

Fold 1
Epoch 42/500, Validation Loss: 0.6367895603179932, Early stopping triggered after 5 epochs.
Fold 2
Epoch 40/500, Validation Loss: 0.3057628870010376, Early stopping triggered after 5 epochs.
Fold 3
Epoch 500/500, Validation Loss: 2117.37255859375, Improvement found, counter reset to 0
Fold 4
Epoch 39/500, Validation Loss: 0.4796893894672394, Early stopping triggered after 5 epochs.
Fold 5
Epoch 24/500, Validation Loss: 1.1123477220535278, Early stopping triggered after 5 epochs.
Fold score (RMSE): 1.0278087854385376
